In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tf_keras.preprocessing.image import ImageDataGenerator
import tf_keras

2024-08-28 22:30:05.466527: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 22:30:05.478079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 22:30:05.489880: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 22:30:05.493049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 22:30:05.503279: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (299, 299))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

full_images, full_labels = load_images_from_folder('Full', 0)
half_images, half_labels = load_images_from_folder('Half', 1)
low_images, low_labels = load_images_from_folder('Low', 2)
no_images, no_labels= load_images_from_folder("Absent", 3)

images = np.concatenate((full_images, half_images, low_images, no_images))
labels = np.concatenate((full_labels, half_labels, low_labels, no_labels))
images = images / 255.0

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [4]:
inception_v3= "https://www.kaggle.com/models/google/inception-v3/TensorFlow2/classification/2"

model= tf_keras.Sequential([
    hub.KerasLayer(
        inception_v3, 
        input_shape= (299, 299, 3), 
        trainable= False
        ),
        tf_keras.layers.Dense(128, activation='relu'),
        tf_keras.layers.Dropout(0.5),
        tf_keras.layers.Dense(4, activation='softmax')
])

with tf.device("/GPU:0"):
    model.compile(
        optimizer= "adam", 
        loss= "sparse_categorical_crossentropy",
        metrics= ["accuracy"]
    )

model.summary()

I0000 00:00:1724864497.797626   27815 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864497.822729   27815 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864497.822781   27815 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864497.825661   27815 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864497.825709   27815 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              23853833  
                                                                 
 dense (Dense)               (None, 128)               128256    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 23982605 (91.49 MB)
Trainable params: 128772 (503.02 KB)
Non-trainable params: 23853833 (91.00 MB)
_________________________________________________________________


In [5]:
with tf.device("/GPU:0"):
    history= model.fit(
        datagen.flow(X_train, y_train, batch_size= 32),
        epochs= 5, 
        validation_data= (X_test, y_test)
    )

Epoch 1/5


2024-08-28 22:31:42.573797: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
W0000 00:00:1724864502.630526   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.696649   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.702194   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.704161   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.706688   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.708920   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.711248   28481 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864502.713504   28481 gpu_t

 1/15 [=>............................] - ETA: 1:26 - loss: 2.1920 - accuracy: 0.1875

W0000 00:00:1724864506.073015   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.074853   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.077032   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.078684   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.080566   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.082616   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.084949   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.087164   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864506.089443   28480 gp

 2/15 [===>..........................] - ETA: 28s - loss: 1.7500 - accuracy: 0.3016 

...

15/15 [==============================] - ETA: 0s - loss: 0.5260 - accuracy: 0.7996

W0000 00:00:1724864512.352054   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.353780   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.355444   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.357068   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.358666   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.360254   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.362797   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.364862   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864512.367397   28480 gp

15/15 [==============================] - 14s 590ms/step - loss: 0.5260 - accuracy: 0.7996 - val_loss: 0.0397 - val_accuracy: 1.0000
Epoch 2/5


W0000 00:00:1724864514.176026   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.177142   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.178227   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.179104   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.179998   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.181206   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.191888   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.192853   28480 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864514.193760   28480 gp

15/15 [==============================] - 4s 259ms/step - loss: 0.0496 - accuracy: 0.9937 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 3/5
15/15 [==============================] - 4s 257ms/step - loss: 0.0234 - accuracy: 0.9958 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 4/5
15/15 [==============================] - 4s 266ms/step - loss: 0.0148 - accuracy: 0.9979 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 5/5
15/15 [==============================] - 4s 265ms/step - loss: 0.0192 - accuracy: 0.9979 - val_loss: 0.0016 - val_accuracy: 1.0000


In [6]:
model.save("H5/inception_v3.h5")

/home/manankhatri/.local/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 0s 85ms/step - loss: 0.0016 - accuracy: 1.0000
Test Accuracy: 100.00%
